In [21]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

df_full_user = pd.read_csv("../preprosscedDataSet.csv")
df_games = pd.read_csv("../full_api_preprocessing.csv")

In [33]:
df_user = pd.DataFrame(df_full_user[["user_id","03_hltb_id"," time_is_play", "06_time_100"]])
df_user.shape

(108435, 4)

In [40]:
df_user["rating_reco"] = df_user[" time_is_play"]/df_user["06_time_100"]
df_user["rating_reco"] = df_user["rating_reco"].replace(-np.inf,0)
df_user["rating_reco"] = df_user["rating_reco"].replace(np.inf,0)
df_user.pop(" time_is_play")
df_user.pop("06_time_100")
df_user = df_user.dropna()
df_user.shape

C:\Users\ichak\AppData\Local\Temp\ipykernel_18280\4185542749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user["rating_reco"] = df_user[" time_is_play"]/df_user["06_time_100"]
C:\Users\ichak\AppData\Local\Temp\ipykernel_18280\4185542749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user["rating_reco"] = df_user["rating_reco"].replace(-np.inf,0)
C:\Users\ichak\AppData\Local\Temp\ipykernel_18280\4185542749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

(97924, 3)

In [41]:
df_user

,user_id,03_hltb_id,rating_reco
0,5250,368.0,0.000108
1,76767,368.0,0.000033
2,298950,368.0,0.000048
3,975449,368.0,0.000198
4,1364546,368.0,0.000035
...,...,...,...
108427,242325247,21210.0,0.000000
108430,243664576,28069.0,0.000000
108431,245314153,13305.0,0.000000
108433,247464119,18591.0,0.000000
